In [114]:
%%capture
!pip install streamlit Flask requests

In [115]:
import requests
import pandas as pd

# proposition de code pour tous les indicateurs

In [116]:
indicators = [
    "EMP_TEMP_SEX_AGE_NB_A",  # Employment by sex and age – ILO modelled estimates, Annual
    "EMP_NAGR_SEX_AGE_NB_A",  # Employment in agriculture by sex and age – ILO modelled estimates, Annual
    "UNE_TUNE_SEX_AGE_RT_A", # (Taux de chômage des jeunes) Unemployment rate by sex and age – ILO modelled estimates, Annual
    "EAP_DWAP_SEX_AGE_EDU_RT_A",  # Employment-to-population ratio by sex, age and educational attainment – ILO modelled estimates, Annual
    "EMP_NIFL_SEX_AGE_GEO_RT_A", #(Taux d'emploi informel) Employment by sex, age, and geographical location – ILO modelled estimates, Annual
    "SDG_0841_SEX_ECO_RT_A", # Average hourly earnings of employees, by sex, economic activity and occupation – ILO modelled estimates, Annual
    #********************************************
      #"EMP_NIFL_SEX_INS_MTS_RT_A",  # Employment by sex, status in employment, and migrant status - ILO modelled estimates, Annual
       #"EMP_NIFL_SEX_INS_MTS_RT_A",  # Employment by sex, status in employment, and migrant status - ILO modelled estimates, Annual
      # "EMP_PIFL_SEX_RT_A", # Employment by sex and place of residence – ILO modelled estimates, Annual

    # Unemployment to employment ratio by sex and age – ILO modelled estimates, Annual
    "UNE_2EAP_SEX_AGE_RT_A",
    # Time-related underemployment rate by sex, and disability status – ILO modelled estimates, Annual
    #"TRU_DEMP_SEX_DSB_RT_A",
    # Labour share of GDP, comprising wages and salaries and social protection expenditure, by sex and economic activity (SDG indicator 10.4.1) – ILO modelled estimates, Annual
    #"SDG_1041_NOC_RT_A",
    # Proportion of informal employment in non-agricultural employment, by sex (SDG indicator 9.2.2) – ILO modelled estimates, Annual
    "SDG_0922_NOC_RT_A",
    # Annual growth rate of real GDP per employed person (SDG indicator 8.2.1) – ILO modelled estimates, Annual
    "SDG_0821_NOC_RT_A",
    # Proportion of population below international poverty line, by sex and social protection coverage (SDG indicator 1.3.1) – ILO modelled estimates, Annual
    "SDG_0131_SEX_SOC_RT_A",
    # Proportion of population living below the national poverty line, by sex and age (SDG indicator 1.1.1) – ILO modelled estimates, Annual
    "SDG_0111_SEX_AGE_RT_A",
    # Labour force participation rate by sex, type of household, and geographical location – ILO modelled estimates, Annual
    #"GED_PLFP_SEX_HHT_GEO_RT_A",
    # Proportion of employed population below a poverty line, by sex, type of household, and geographical location – ILO modelled estimates, Annual
    "GED_PEPR_SEX_HHT_GEO_RT_A",
    #"EMP_TEMP_SEX_EST_DSB_NB_A",
    #'EMP_TEMP_SEX_EDU_GEO_NB_A',
    'GDP_2TOT_NOC_NB_A', #gdp
    'GDP_205U_NOC_NB_A', #outpout per worker
    #'SDG_0131_SEX_SOC_RT_A',
    #'CPI_XCPI_COI_RT_A', # inflation base 100=2017
    'POP_2POP_SEX_AGE_NB_A', #population
]

indicators=list(set(indicators))
print(len(indicators))
indicators


15


['UNE_TUNE_SEX_AGE_RT_A',
 'SDG_0111_SEX_AGE_RT_A',
 'EMP_NIFL_SEX_AGE_GEO_RT_A',
 'EAP_DWAP_SEX_AGE_EDU_RT_A',
 'SDG_0922_NOC_RT_A',
 'UNE_2EAP_SEX_AGE_RT_A',
 'GED_PEPR_SEX_HHT_GEO_RT_A',
 'GDP_2TOT_NOC_NB_A',
 'SDG_0131_SEX_SOC_RT_A',
 'SDG_0821_NOC_RT_A',
 'EMP_TEMP_SEX_AGE_NB_A',
 'POP_2POP_SEX_AGE_NB_A',
 'EMP_NAGR_SEX_AGE_NB_A',
 'SDG_0841_SEX_ECO_RT_A',
 'GDP_205U_NOC_NB_A']

## downloading the data

In [117]:
def load_data(indicators):
  import pandas as pd

  all_data = pd.DataFrame()
  # le chargement des data prend 13m 55s soit 14 minutes
  for indicator in indicators:
      url=f'https://rplumber.ilo.org/data/indicator/?id={indicator}&type=label&region=africa&format=.xlsx'
      #url = f"https://rplumber.ilo.org/data/indicator/?id={indicator}&ref_area={country}&freq=A"  # Adjust freq as needed
      try:
          indicator_data = pd.read_excel(url)
          #country_data['Pays'] = country
          indicator_data['indicator'] = indicator
          all_data = pd.concat([all_data, indicator_data], ignore_index=True)
      except Exception as e:
          print(f"Erreur pour le pays {indicator}: {e}")
  return all_data

In [118]:
# Afficher les premières lignes du DataFrame global
#all_data = load_data(indicators)
#all_data.head()

In [119]:
#all_data.shape

######GPS

In [120]:
%%capture
!pip install countryinfo wbdata

In [121]:
#coordonnees gps
from countryinfo import CountryInfo
import numpy as np
def get_country_coordinates(country_name):
    try:
        country = CountryInfo(country_name)
        coordinates = country.latlng()
        return coordinates[0], coordinates[1]  # Latitude, Longitude
    except KeyError:
        return None, None  # Handle cases where country info is not available

def add_gps_columns(all_data):

  # Create a dictionary to map old names to new names
  country_name_mapping = {
    'Sao Tome and Principe': 'São Tomé and Príncipe',
    'Tanzania, United Republic of': 'Tanzania',
    'Eswatini': 'Swaziland',
    'Cabo Verde': 'Cape Verde',
    'Gambia': 'The Gambia',
    'Congo': 'Republic of the Congo',
    'Congo, Democratic Republic of the': 'Democratic Republic of the Congo',
    "Côte d'Ivoire": "Ivory Coast"
  }

  # Use the replace method with the mapping dictionary
  all_data['ref_area.label'] = all_data['ref_area.label'].replace(country_name_mapping)
  africa_countries= np.unique(all_data['ref_area.label'].values)

  lat=[]
  lon=[]

  for country in africa_countries:
    latitude, longitude = get_country_coordinates(country)
    if latitude is not None:
        #print(f"{country}: Latitude = {latitude}, Longitude = {longitude}")
        lat.append(latitude)
        lon.append(longitude)
    else:
        print(f"Could not retrieve coordinates for {country}")
        lat.append(None)
        lon.append(None)
  df=pd.DataFrame({'ref_area.label':africa_countries,'lat':lat,'lon':lon})
  all_data=pd.merge(all_data, df, on='ref_area.label', how='left')
#****************************************************************************************
    #adding region columns
#************************************************************************************
  # list of country names
  country_names = ['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso',
        'Burundi', 'Cape Verde', 'Cameroon', 'Central African Republic',
        'Chad', 'Comoros', 'Republic of the Congo', 'Democratic Republic of the Congo',
        "Ivory Coast", 'Djibouti', 'Egypt', 'Equatorial Guinea',
        'Eritrea', 'Swaziland', 'Ethiopia', 'Gabon', 'The Gambia', 'Ghana',
        'Guinea', 'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia', 'Libya',
        'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius',
        'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda',
        'São Tomé and Príncipe', 'Senegal', 'Seychelles', 'Sierra Leone',
        'Somalia', 'South Africa', 'South Sudan', 'Sudan',
        'Tanzania', 'Togo', 'Tunisia', 'Uganda',
        'Zambia', 'Zimbabwe']

  # Corresponding list of regions
  regions = ['North', 'Central', 'West', 'Southern', 'West', 'East', 'West',
            'Central', 'Central', 'Central', 'East', 'Central', 'Central',
            'West', 'East', 'North', 'Central', 'East', 'Southern', 'East',
            'Central', 'West', 'West', 'West', 'West', 'East', 'Southern',
            'West', 'North', 'East', 'Southern', 'West', 'West', 'East',
            'North', 'Southern', 'Southern', 'West', 'West', 'East',
            'Central', 'West', 'East', 'West', 'East', 'Southern', 'East',
            'North', 'East', 'West', 'North', 'East', 'Southern',
            'Southern']

  # Create a Pandas DataFrame
  africa_regions = pd.DataFrame({'ref_area.label': country_names, 'Region': regions})

  # Filter the DataFrame to include only African countries
  all_data_africa = all_data[all_data['ref_area.label'].isin(africa_regions['ref_area.label'])].copy()

  # Add the 'Region' column based on the dictionary
  all_data_africa = all_data_africa.merge(africa_regions, on='ref_area.label', how='left')

  return all_data_africa

#Preprocessing

##Statistics of missing data

---



* missing data per key variable

In [122]:
def missing_stat(all_data_africa):
    # nombre de donnees manquante par annee 'time'
  missing_data_per_year = all_data_africa.groupby('time')['obs_value'].agg(['size', lambda x: x.isnull().sum()])
  missing_data_per_year.columns = ['total_expected', 'missing_count']
  missing_data_per_year['missing_percentage'] = (missing_data_per_year['missing_count'] / missing_data_per_year['total_expected']) * 100
  # Format the percentage to two decimal places
  missing_data_per_year['missing_percentage'] = missing_data_per_year['missing_percentage'].map('{:.2f}'.format)
  # Filter and sort
  missing_data_per_year = missing_data_per_year[missing_data_per_year['missing_count'] != 0].sort_values('missing_count', ascending=False)
#``````````````````````````````````````````````````````````````````
    # nombre de donnees manquante par annee 'time' et par 'indicator.label'
  missing_data_per_year_indicator = all_data_africa.groupby(['time', 'indicator.label'])['obs_value'].agg(['size', lambda x: x.isnull().sum()]).reset_index()
  missing_data_per_year_indicator.columns = ['time', 'indicator.label', 'total_expected', 'missing_count'] # rename columns
  # Calculate the percentage of missing values
  missing_data_per_year_indicator['missing_percentage'] = (missing_data_per_year_indicator['missing_count'] / missing_data_per_year_indicator['total_expected']) * 100
  missing_data_per_year_indicator = missing_data_per_year_indicator[missing_data_per_year_indicator['missing_count'] != 0].sort_values('missing_count', ascending=False)
  #missing_data_per_year_indicator

    # Filter the DataFrame for years between 1990 and 2023
  filtered_data = all_data_africa[(all_data_africa['time'] >= 2015) & (all_data_africa['time'] <= 2023)]
#``````````````````````````````````````````````````````````
  # Calculate the percentage of missing values for each variable
  missing_percentage = filtered_data.isnull().sum() * 100 / len(filtered_data)
  missing_percentage.sort_values(ascending=False)
  # Filter variables with more than 50% missing values
  #variables_to_drop = []#missing_percentage[missing_percentage > 60].index
  # Drop the variables
  #filtered_data = filtered_data.drop(columns=variables_to_drop,axis=1)
  return filtered_data

* Nombre de donnees manquante par annee 'time' et par 'indicator.label'

In [123]:
def methode_valeurs_manquantes(df, method='forward_fill', limit=2):
    if method == 'forward_fill':
        return df.ffill(limit=limit)
    elif method == 'backward_fill':
        return df.bfill(limit=limit)
    elif method == 'linear_interpolation':
        return df.interpolate(method='linear', limit_direction='both')
    else:
        raise ValueError("Méthode non reconnue. Choisissez 'ffill', 'bfill' ou 'interpolate'.")
#*****************************************************************************************************

def detect_and_treat_missing_values_by_country_variable(panel_df, method='forward_fill', limit=2):
  # Définir les colonnes contenant les variables
  variable_cols = panel_df.drop(['ref_area.label', 'time', 'lat', 'lon', 'Region'], axis=1).columns  # variable1 à variable217

  # Traiter les valeurs manquantes par pays
  for pays, groupe in panel_df.groupby('ref_area.label'):
      # Appliquer la méthode choisie à chaque groupe
      panel_df.loc[panel_df['ref_area.label'] == pays, variable_cols] = methode_valeurs_manquantes(groupe[variable_cols], method='forward_fill')
  return panel_df

In [124]:
def detect_and_treat_missing_values_by_country_variable2(df, method='forward_fill', limit=2):
    """
    Detects and treats missing values in a pandas DataFrame, grouped by country and variable.

    Args:
        df: The input DataFrame with columns 'ref_area.label', 'indicator.label', 'time', and 'obs_value'.
        method: The method for missing value imputation ('forward_fill', 'backward_fill', or 'linear_interpolation').
        limit: The maximum number of consecutive NaN values to fill (for 'forward_fill' and 'backward_fill').

    Returns:
        A new DataFrame with missing values treated.
    """
    # Supprimer les doublons
    df = df.drop_duplicates()

    def apply_method(group):
        if method == 'forward_fill':
            group['obs_value'] = group['obs_value'].ffill(limit=limit)
        elif method == 'backward_fill':
            group['obs_value'] = group['obs_value'].bfill(limit=limit)
        elif method == 'linear_interpolation':
            group['obs_value'] = group['obs_value'].interpolate(method='linear', limit_direction='both')
        elif method == 'mean':
          group['obs_value'] = group['obs_value'].fillna(group['obs_value'].mean())
        else:
            print(f"Warning: Unknown method '{method}'. No missing value treatment applied for {group.name[0]} - {group.name[1]}.")
                # Appliquer l'interpolation linéaire pour les valeurs restantes
        #group['obs_value'] = group['obs_value'].interpolate(method='linear_interpolation', limit_direction='both')
        return group

    df_treated = df.groupby(['ref_area.label', 'indicator.label'], group_keys=False).apply(apply_method) #

    return df_treated

#*******************************************************************************************************************************************
def treat_outliers(df):
    """Traite les valeurs aberrantes en supprimant les valeurs extrêmes."""

    return df


#****************************************************************************

#*********************************************************************************************************************************
def remove_columns_with_high_missing_percentage(df, threshold=0.6):
    missing_percentage = df.isnull().sum() / len(df)
    columns_to_drop = missing_percentage[missing_percentage > threshold].index
    return df.drop(columns=columns_to_drop)


In [125]:
import re

def simplify_and_rename(column_name):
    """Simplifies and renames a column name, handling age ranges."""
    parts = re.split(r'[_\(\)\:\s]+', column_name)  # Split by underscores, parentheses, colons, and spaces

    # Essential parts: Indicator, Sex, Age (if available)
    indicator = parts[0]
    if 'Sex' in parts:
        sex_index = parts.index('Sex')
        sex = parts[sex_index + 1]
    else:
        sex = 'nan'

    # Extract age range using regex
    age_match = re.search(r'\)\:\s*([\w\-\+]+)', column_name)  # Find text after closing parenthesis and colon
    if age_match:
        age = age_match.group(1)
    else:
        age = 'nan'
    # Common indicator abbreviations
    indicator_mapping = {
        'Employment': 'EMP',
        'Gross': 'GDP',  # Assuming 'Gross' implies GDP
        'Informal': 'InformalRate',
        'Labour': 'LaborForce',
        'Output': 'OutputPerWorker',
        'Population': 'POP',
        'Prime-age': 'PrimeAgeEMP',
        'SDG': 'SDG',
        'Unemployment': 'UnempRate',

    }

    indicator = indicator_mapping.get(indicator, indicator)  # Use abbreviation if found

    # Construct the new name
    new_name = f"{indicator}_{sex}_{age}"
    new_name = new_name.replace('thousands', '') # remove 'thousands'
    new_name = new_name.replace('rate', '') # remove 'rate'
    new_name = new_name.replace(' ', '') # remove ' '
    new_name = new_name.replace('(', '') # remove '('
    new_name = new_name.replace(')', '') # remove ')'
    new_name = new_name.replace('%', '') # remove '%'
    return new_name


In [126]:
def rename_all_columns(df1, df2,df3):
  # Create a dictionary to map old names to new names
  rename_mapping = {
      'Employment by sex and age (thousands)': 'Employment',
      'Gross Domestic Product (GDP constant 2017 international $ at PPP) -- ILO modelled estimates, Nov. 2023 (millions of 2017 PPP dollars)': 'GDP',
      'Informal employment rate by sex, age and rural / urban areas (%)': 'Informal Employment Rate',
      'Labour force participation rate by sex, age and education (%)': 'Labor Force Participation Rate',
      'Output per worker (GDP constant 2015 US $) -- ILO modelled estimates, Nov. 2023': 'Output per Worker',
      'Population by sex and age -- UN estimates and projections, July 2022 (thousands)': 'Population',
      'Prime-age employment-to-population ratio by sex, household type and rural / urban areas (%)': 'Prime-Age Employment-to-Population Ratio',
      'Region': 'Region',
      'SDG indicator 1.1.1 - Working poverty rate (percentage of employed living below US$2.15 PPP) (%)': 'Working Poverty Rate',
      'SDG indicator 1.3.1 - Proportion of population covered by social protection floors/systems (%)': 'Social Protection Coverage',
      'SDG indicator 8.2.1 - Annual growth rate of output per worker (GDP constant 2017 international $ at PPP) (%)': 'Output per Worker Growth Rate',
      'SDG indicator 1.1.1 - Working poverty rate (percentage of employed living below US$2.15 PPP) (%)': 'Working Poverty Rate',
      'SDG indicator 1.3.1 - Proportion of population covered by social protection floors/systems (%)': 'Social Protection Coverage',
      'SDG indicator 8.2.1 - Annual growth rate of output per worker (GDP constant 2017 international $ at PPP) (%)': 'Output per Worker Growth Rate',
      'SDG indicator 9.2.2 - Manufacturing employment as a proportion of total employment (%)': 'Manufacturing Employment Share',
      'Unemployment rate by sex and age -- ILO modelled estimates, May 2024 (%)': 'Unemployment Rate',

  }

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  rename_mapping_sexe = {

      'Employment by sex and age (thousands)_Sex: Female': 'Employment_Female',
      'Employment by sex and age (thousands)_Sex: Male': 'Employment_Male',
      'Employment by sex and age (thousands)_Sex: Total': 'Employment_Total',
      'Informal employment rate by sex, age and rural / urban areas (%)_Sex: Female': 'Informal Employment Rate_Female',
      'Informal employment rate by sex, age and rural / urban areas (%)_Sex: Male': 'Informal Employment Rate_Male',
      'Informal employment rate by sex, age and rural / urban areas (%)_Sex: Total': 'Informal Employment Rate_Total',
      'Labour force participation rate by sex, age and education (%)_Sex: Female': 'Labor Force Participation Rate_Female',
      'Labour force participation rate by sex, age and education (%)_Sex: Male': 'Labor Force Participation Rate_Male',
      'Labour force participation rate by sex, age and education (%)_Sex: Total': 'Labor Force Participation Rate_Total',
      'Population by sex and age -- UN estimates and projections, July 2022 (thousands)_Sex: Female': 'Population_Female',
      'Population by sex and age -- UN estimates and projections, July 2022 (thousands)_Sex: Male': 'Population_Male',
      'Population by sex and age -- UN estimates and projections, July 2022 (thousands)_Sex: Total': 'Population_Total',
      'Prime-age employment-to-population ratio by sex, household type and rural / urban areas (%)_Sex: Female': 'Prime-Age Employment-to-Population Ratio_Female',
      'Prime-age employment-to-population ratio by sex, household type and rural / urban areas (%)_Sex: Male': 'Prime-Age Employment-to-Population Ratio_Male',
      'Prime-age employment-to-population ratio by sex, household type and rural / urban areas (%)_Sex: Total': 'Prime-Age Employment-to-Population Ratio_Total',
      'SDG indicator 1.1.1 - Working poverty rate (percentage of employed living below US$2.15 PPP) (%)_Sex: Female': 'Working Poverty Rate_Female',
      'SDG indicator 1.1.1 - Working poverty rate (percentage of employed living below US$2.15 PPP) (%)_Sex: Male': 'Working Poverty Rate_Male',
      'SDG indicator 1.1.1 - Working poverty rate (percentage of employed living below US$2.15 PPP) (%)_Sex: Total': 'Working Poverty Rate_Total',
      'SDG indicator 1.3.1 - Proportion of population covered by social protection floors/systems (%)_Sex: Female': 'Social Protection Coverage_Female',
      'SDG indicator 1.3.1 - Proportion of population covered by social protection floors/systems (%)_Sex: Male': 'Social Protection Coverage_Male',
      'SDG indicator 1.3.1 - Proportion of population covered by social protection floors/systems (%)_Sex: Total': 'Social Protection Coverage_Total',
      'Unemployment rate by sex and age -- ILO modelled estimates, May 2024 (%)_Sex: Female': 'Unemployment Rate_Female',
      'Unemployment rate by sex and age -- ILO modelled estimates, May 2024 (%)_Sex: Male': 'Unemployment Rate_Male',
      'Unemployment rate by sex and age -- ILO modelled estimates, May 2024 (%)_Sex: Total': 'Unemployment Rate_Total',

  }

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  columns_to_rename = [col for col in df2.columns
                      if col not in ['ref_area.label', 'lat', 'lon', 'Region']]
  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Rename columns in all three DataFrames
  df1 = df1.rename(columns=rename_mapping_sexe)
  df2 = df2.rename(columns=lambda x: simplify_and_rename(x) if x in columns_to_rename else x)
  #df2=df2.rename(columns=lambda x: simplify_and_rename(x))
  df3 = df3.rename(columns=rename_mapping)
  df=pd.concat([df1, df2, df3], axis=1)
  df = df.loc[:, ~df.columns.duplicated()]
  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  return df

In [127]:

def transpose_to_panel(df):
  # 1. Create the first DataFrame (indicator + sex)
  df['variable1'] = df['indicator.label'] + '_' + df['sex.label']
  panel_df1 = df[['ref_area.label', 'time', 'lat', 'lon', 'Region', 'variable1', 'obs_value']]
  panel_df1 = panel_df1.pivot_table(index=['ref_area.label', 'time', 'lat', 'lon', 'Region'],
                                   columns='variable1',
                                   values='obs_value').reset_index()

  # 2. Create the second DataFrame (indicator + sex + classif1)
  df['variable2'] = df['indicator.label'] + '_' + df['sex.label'] + '_' + df['classif1.label']
  panel_df2 = df[['ref_area.label', 'time', 'lat', 'lon', 'Region', 'variable2', 'obs_value']]
  panel_df2 = panel_df2.pivot_table(index=['ref_area.label', 'time', 'lat', 'lon', 'Region'],
                                   columns='variable2',
                                   values='obs_value').reset_index()

  # 3. Create the third DataFrame (indicator + sex + classif1 + classif2)
  df['variable3'] = df['indicator.label'] #+ '_' + df['sex.label'] + '_' + df['classif1.label'] + '_' + df['classif2.label']
  panel_df3 = df[['ref_area.label', 'time', 'lat', 'lon', 'Region', 'variable3', 'obs_value']]
  panel_df3 = panel_df3.pivot_table(index=['ref_area.label', 'time', 'lat', 'lon', 'Region'],
                                   columns='variable3',
                                   values='obs_value').reset_index()

  # 4. Check if DataFrames have the same shape (excluding index)
  same_shape = panel_df1.shape[1:] == panel_df2.shape[1:] == panel_df3.shape[1:]
  print(f"DataFrames have the same shape (excluding index): {same_shape}")

  return panel_df1, panel_df2, panel_df3

  #***********************************************************************************************************************************************

#**********************************************************************************************************************************

def main(indicators):
    """Fonction principale pour charger, nettoyer, et transformer les données."""
    df = load_data(indicators)
    df1 = add_gps_columns(df)
    df2=missing_stat(df1)
    #df_3=detect_and_treat_missing_values_by_country_variable2(df2, method='forward_fill', limit=2)
    #df_3=detect_and_treat_missing_values_by_country_variable2(df_3, method='linear_interpolation', limit=2)
    #df = treat_outliers(df_3)

    df_panel1 = transpose_to_panel(df2)[0]
    df_panel2 = transpose_to_panel(df2)[1]
    df_panel3 = transpose_to_panel(df2)[2]
    df_panel_rename=rename_all_columns(df_panel1,df_panel2,df_panel3)
    df_final=remove_columns_with_high_missing_percentage(df_panel_rename)
    #df_final=detect_and_treat_missing_values_by_country_variable(df_panel, method='forward_fill', limit=2)
    df_final=df_panel_rename.rename(columns={'ref_area.label':'pays','time':'annee'})
    return df_final

In [128]:
# Example usage (assuming 'df' is your cleaned dataframe)
panel_df = main(indicators)

<ipython-input-127-90fad8584495>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['variable1'] = df['indicator.label'] + '_' + df['sex.label']
<ipython-input-127-90fad8584495>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['variable2'] = df['indicator.label'] + '_' + df['sex.label'] + '_' + df['classif1.label']
<ipython-input-127-90fad8584495>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

DataFrames have the same shape (excluding index): False


<ipython-input-127-90fad8584495>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['variable1'] = df['indicator.label'] + '_' + df['sex.label']
<ipython-input-127-90fad8584495>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['variable2'] = df['indicator.label'] + '_' + df['sex.label'] + '_' + df['classif1.label']
<ipython-input-127-90fad8584495>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

DataFrames have the same shape (excluding index): False


<ipython-input-127-90fad8584495>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['variable1'] = df['indicator.label'] + '_' + df['sex.label']
<ipython-input-127-90fad8584495>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['variable2'] = df['indicator.label'] + '_' + df['sex.label'] + '_' + df['classif1.label']
<ipython-input-127-90fad8584495>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

DataFrames have the same shape (excluding index): False


In [129]:
np.unique(panel_df.columns)

array(['EMP_Female_15+', 'EMP_Female_15-19', 'EMP_Female_15-24',
       'EMP_Female_15-64', 'EMP_Female_20-24', 'EMP_Female_25+',
       'EMP_Female_25-29', 'EMP_Female_25-34', 'EMP_Female_25-54',
       'EMP_Female_30-34', 'EMP_Female_35-39', 'EMP_Female_35-44',
       'EMP_Female_40-44', 'EMP_Female_45-49', 'EMP_Female_45-54',
       'EMP_Female_50-54', 'EMP_Female_55-59', 'EMP_Female_55-64',
       'EMP_Female_60-64', 'EMP_Female_65+', 'EMP_Female_Total',
       'EMP_Male_15+', 'EMP_Male_15-19', 'EMP_Male_15-24',
       'EMP_Male_15-64', 'EMP_Male_20-24', 'EMP_Male_25+',
       'EMP_Male_25-29', 'EMP_Male_25-34', 'EMP_Male_25-54',
       'EMP_Male_30-34', 'EMP_Male_35-39', 'EMP_Male_35-44',
       'EMP_Male_40-44', 'EMP_Male_45-49', 'EMP_Male_45-54',
       'EMP_Male_50-54', 'EMP_Male_55-59', 'EMP_Male_55-64',
       'EMP_Male_60-64', 'EMP_Male_65+', 'EMP_Male_Total',
       'EMP_Total_15+', 'EMP_Total_15-19', 'EMP_Total_15-24',
       'EMP_Total_15-64', 'EMP_Total_20-24', 'EMP_Tota

In [130]:
# Calculate the percentage of missing values for each variable
missing_percentage = panel_df.isnull().sum() * 100 / len(panel_df)

# Create a DataFrame from the missing percentage data
missing_data_df = pd.DataFrame({'Variable': missing_percentage.index, 'Missing Percentage': missing_percentage.values})

# Sort the DataFrame by missing percentage in descending order
missing_data_df = missing_data_df.sort_values(by='Missing Percentage', ascending=False)

# Print the sorted DataFrame
missing_data_df[0:200]

,Variable,Missing Percentage
243,SDG_Female_nan,96.494845
244,SDG_Male_nan,96.494845
231,PrimeAgeEMP_Female_nan,86.597938
232,PrimeAgeEMP_Male_nan,86.597938
233,PrimeAgeEMP_Total_nan,86.597938
...,...,...
248,UnempRate_Female_25+,1.649485
226,POP_Total_60-64,1.649485
247,UnempRate_Female_15-24,1.649485
246,UnempRate_Female_15+,1.649485


saving the data

In [1]:
# # mount google drive and specify folder path
from google.colab import drive
#drive.mount('/content/drive', force_remount=True)
#folder_path = '/content/drive/MyDrive/Colab Notebooks/board/'
#panel_df.to_excel(folder_path+'baseFINALE_panel.xlsx',index=False)

In [132]:
np.unique(panel_df.columns)

array(['EMP_Female_15+', 'EMP_Female_15-19', 'EMP_Female_15-24',
       'EMP_Female_15-64', 'EMP_Female_20-24', 'EMP_Female_25+',
       'EMP_Female_25-29', 'EMP_Female_25-34', 'EMP_Female_25-54',
       'EMP_Female_30-34', 'EMP_Female_35-39', 'EMP_Female_35-44',
       'EMP_Female_40-44', 'EMP_Female_45-49', 'EMP_Female_45-54',
       'EMP_Female_50-54', 'EMP_Female_55-59', 'EMP_Female_55-64',
       'EMP_Female_60-64', 'EMP_Female_65+', 'EMP_Female_Total',
       'EMP_Male_15+', 'EMP_Male_15-19', 'EMP_Male_15-24',
       'EMP_Male_15-64', 'EMP_Male_20-24', 'EMP_Male_25+',
       'EMP_Male_25-29', 'EMP_Male_25-34', 'EMP_Male_25-54',
       'EMP_Male_30-34', 'EMP_Male_35-39', 'EMP_Male_35-44',
       'EMP_Male_40-44', 'EMP_Male_45-49', 'EMP_Male_45-54',
       'EMP_Male_50-54', 'EMP_Male_55-59', 'EMP_Male_55-64',
       'EMP_Male_60-64', 'EMP_Male_65+', 'EMP_Male_Total',
       'EMP_Total_15+', 'EMP_Total_15-19', 'EMP_Total_15-24',
       'EMP_Total_15-64', 'EMP_Total_20-24', 'EMP_Tota